In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Importing dependencies

import pandas as pd
import numpy as np
import os
import re
from datetime import datetime

from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from sklearn.pipeline import make_pipeline as pipe
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer as LemmaT

from xgboost.sklearn import XGBClassifier as XGB
from sklearn.ensemble import RandomForestClassifier as RF, AdaBoostClassifier as ADA, VotingClassifier as Vote, StackingClassifier as Stack
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from transformers import pipeline

import tensorflow as tf
from tensorflow.keras import Sequential,layers
%matplotlib inline
import fasttext

import tensorflow_hub as hub
# import tensorflow_text as text


2021-10-08 08:38:37.881993: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-08 08:38:37.882106: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
#reading csv

import pandas as pd
import numpy as np

train_data=pd.read_csv("../input/intermediate-nlp-mlbuzz/intermediate/dataset/train.csv")
train_data.head()

,File_name,Industry,Sub-category
0,32577.txt,Power Industry,Green Energy
1,12871.txt,Health Industry,Diagnostic
2,10812.txt,Health Industry,Pharma
3,21494.txt,Manufacturing Industry,Production
4,3744.txt,Automobile Industry,Heavy Vehicles


In [4]:
# train_data.info()

In [5]:
# reading the articles




en_stops=set(stopwords.words('english'))

folder_train="../input/intermediate-nlp-mlbuzz/intermediate/dataset/train"

train_data["Article"]=[""]*len(train_data)

lemmaT=LemmaT()

mx=0

for i in range(len(train_data)):
    rel_path=train_data["File_name"].iloc[i]
    fpath=os.path.join(folder_train,rel_path)
    with open(fpath,'r') as f:
        art=f.read()
        proc_art=''
        
        #preprocessing: Removing stop words and special characters
        
        art=re.sub('[^A-Za-z\s]',' ',art)  #removing all non alphanumeric character with space
        
        for wd in art.split():
            wd=wd.strip()
            if wd not in en_stops:
                wd=wd.lower()
                wd=lemmaT.lemmatize(wd)
                if len(wd)>1:
                    proc_art+=wd+" "

        proc_art=proc_art.strip()
        train_data["Article"].iloc[i]=proc_art
        
        mx=max(mx,len(proc_art))

print("maxlen:",mx)

train_data.head()
        
        
       
    

maxlen: 6472


,File_name,Industry,Sub-category,Article
0,32577.txt,Power Industry,Green Energy,natural gas produce around half emission coal ...
1,12871.txt,Health Industry,Diagnostic,sanctuary santa maria della steccata in bc cit...
2,10812.txt,Health Industry,Pharma,in pure bismarck system access insurance seen ...
3,21494.txt,Manufacturing Industry,Production,voting right volkswagen integrating scania man...
4,3744.txt,Automobile Industry,Heavy Vehicles,lb required upper stage payload perform final ...


In [6]:
#intermediate save
train_data.to_csv("processed_data.csv",index=False)

In [7]:
train_data["Article"].iloc[5]

'household without electricity typically use kerosene lamp lighting creates toxic fume wind turbine also placed ocean wind steadier stronger land construction maintenance cost higher the positive negative made different material positive exhibiting reduction potential negative oxidation potential other effort included formation research facility united state seri nrel japan nedo germany fraunhofer ise ej yr energy used residential building overall energy investment largely economical decision better make investment decision based systematical evaluation option operational improvement energy efficiency onsite generation energy storage manufacturing solar panel requires rare earth element producing low level radioactive waste mining process the industry symbol sdg started january the company small market cap established st pierre current ceo robert hodges current year revenue cr net market valuation cr current share price'

In [8]:
# import pandas as pd
# import numpy as np
# train_data=pd.read_csv("../input/intermediate-nlp-mlbuzz/processed_data.csv")

# train_data.head()

In [9]:
# print(train_data["Article"][3])

In [10]:
# x=train_data["Article"]
# y=train_data["Sub-category"]

# from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
# enc1hot = OneHotEncoder(handle_unknown='ignore')
# enc_ord = OrdinalEncoder()

# y1hot=enc1hot.fit_transform(y.to_numpy().reshape(-1,1))
# y_ord=enc_ord.fit_transform(y.to_numpy().reshape(-1,1))

In [11]:
# # importing bert

# from transformers import DistilBertTokenizer, DistilBertModel

# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# bert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [12]:
# xenc=tokenizer.batch_encode_plus(x,add_special_tokens=True,truncation=True,padding="max_length",max_length=512,return_attention_mask=True,return_tensors='pt')
# print(xenc.shape)

In [13]:
# xenc=xenc['input_ids']

# import torch
# import tensorflow as tf

# xenc = xenc.numpy()


In [14]:
# y1hot_np=y1hot.toarray()
# y1hot_np.shape

In [15]:

# st=["I am going to the moon and drive in rocket","Please dont mind","I am going"]
# enc=tokenizer.encode_plus(st[0],add_special_tokens=True,truncation=True,padding="max_length",max_length=512,return_attention_mask=True,return_tensors='pt')
# b_enc=bert_model(**enc)[0]
# print(b_enc.shape)


In [16]:
# cls_enc=[]
# # nump_cls_enc=np.empty((0,512))
# for i in range(len(x)):
#     st=train_data["Article"].iloc[i]
#     enc=tokenizer.encode_plus(st,add_special_tokens=True,max_length=512,truncation=True,padding="max_length",return_attention_mask=True,return_tensors='pt')
# #     b_enc=bert_model(**enc)[0]
#     cls_enc.append(enc)
# #     np.append(nump_cls_enc,enc,axis=0)

In [17]:
# X=[np.array(i["input_ids"]) for i in cls_enc]
# X=np.array(X).reshape((31779,512))
# print(X.shape,y1hot.shape)

In [18]:
# #test train split

# xtrain,xtest,ytrain,ytest=train_test_split(x,y_ord,test_size=0.4,random_state=7)
# print(len(xtrain),len(xtest),len(ytrain),len(ytest))

In [19]:
# #SVM



# base_model=pipe(CountVectorizer(ngram_range=(1,3)),SVC())
# base_model.fit(x,y_ord)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))


# base_model=pipe(CountVectorizer(ngram_range=(1,3)),TfidfTransformer(),SVC())
# base_model.fit(x,y_ord)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))

In [20]:
# #XGB


# from xgboost.sklearn import XGBClassifier as XGB
# from sklearn.ensemble import RandomForestClassifier as RF, AdaBoostClassifier as ADA, StackingClassifier as Stack
# from sklearn.pipeline import make_pipeline as pipe
# from sklearn.feature_extraction.text import CountVectorizer


# base_model=pipe(CountVectorizer(ngram_range=(1,3)),XGB())
# base_model.fit(xtrain,ytrain)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))

# base_model=pipe(CountVectorizer(ngram_range=(1,3)),TfidfTransformer(),XGB())
# base_model.fit(xtrain,ytrain)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))

In [21]:
# #ADA


# base_model=pipe(CountVectorizer(ngram_range=(1,3)),ADA())
# base_model.fit(xtrain,ytrain)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))


# base_model=pipe(CountVectorizer(ngram_range=(1,3)),TfidfTransformer(),ADA())
# base_model.fit(xtrain,ytrain)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))

In [22]:
# #RF

# base_model=pipe(CountVectorizer(ngram_range=(1,3)),RF())
# base_model.fit(xtrain,ytrain)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))


# base_model=pipe(CountVectorizer(ngram_range=(1,3)),TfidfTransformer(),RF())
# base_model.fit(xtrain,ytrain)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))

In [23]:
# #Naive-Bayes

# base_model=pipe(CountVectorizer(ngram_range=(1,3)),MultinomialNB())
# base_model.fit(xtrain,ytrain)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))

# base_model=pipe(CountVectorizer(ngram_range=(1,3)),TfidfTransformer(),MultinomialNB())
# base_model.fit(xtrain,ytrain)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))

In [24]:
# # #KNN


# base_model=pipe(CountVectorizer(ngram_range=(1,3)),KNeighborsClassifier(n_neighbors=5))
# base_model.fit(xtrain,ytrain)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))

# base_model=pipe(CountVectorizer(ngram_range=(1,3)),TfidfTransformer(),KNeighborsClassifier(n_neighbors=5))
# base_model.fit(xtrain,ytrain)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))


In [25]:
# #Voting


# rf=RF()
# svm=SVC(random_state=7)
# xgb=XGB(random_state=7)
# models=[("XGB",xgb),("SVM",svm)]

# base_model=pipe(CountVectorizer(ngram_range=(1,3)),Vote(models))
# base_model.fit(x,y_ord)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))

# # # base_model=pipe(CountVectorizer(ngram_range=(1,3)),TfidfTransformer(),Vote(models))
# # # base_model.fit(xtrain,ytrain)
# # # ypred=base_model.predict(xtest)
# # # print(acc(ypred,ytest))

In [26]:
# #stacking


# rf=RF()
# xgb=XGB()
# svm=SVC()
# models=[("XGB",xgb),("SVM",svm)]

# base_model=pipe(CountVectorizer(ngram_range=(1,3)),Stack(models,final_estimator=XGB()))
# base_model.fit(x,y_ord)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))


# base_model=pipe(CountVectorizer(ngram_range=(1,3)),TfidfTransformer(),Stack(models))
# base_model.fit(x,y_ord)
# ypred=base_model.predict(xtest)
# print(acc(ypred,ytest))

In [27]:
# print(acc(ypred,ytest))

In [28]:
# #zero-shot

# from transformers import pipeline
# classifier = pipeline("zero-shot-classification",model="facebook/bart-large-mnli")

# candidate_labels = ['Green Energy', 'Non-green Energy', 'Diagnostic', 'Pharma', 'Production', 'Toys', 'Food', 'Mechanical', 'Heavy Vehicles', 'Light Vehicles', 'Hardware', 'Software', 'Security']
# ypred=[]
# for i in range(20):
#     sequence_to_classify = train_data["Article"].iloc[i]
#     prediction=classifier(sequence_to_classify, candidate_labels)
#     labels=prediction['labels']
#     score=prediction['scores']
#     pred=np.argmax(score)
#     pred=labels[pred]
#     ypred.append(pred)
#     print("Actual:",y[i],", Predicted:",pred)
#     if i%10==0:
#         print(i,datetime.now())
    
# print(acc(ypred,y[:20]))

In [29]:
# # all the industry can be directly predicted if sub categories is known
# ind_cat={}
# cat_ind={}
# for i in range(len(train_data)):
#     ind=train_data["Industry"].iloc[i]
#     cat=train_data["Sub-category"].iloc[i]
#     if ind in ind_cat:
#         if cat not in ind_cat[ind]:
#             ind_cat[ind].append(cat)
#     else:
#         ind_cat[ind]=[cat]
        
#     if cat not in cat_ind:
#         cat_ind[cat]=ind
        
# print(ind_cat,cat_ind)

In [30]:
# #DNN

# def dl_nn():
#     model=Sequential()
#     #model.add(layers.BatchNormalization(input_shape=(8,)))
#     model.add(layers.Dense(512,input_shape=(None,1420510),activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
#     model.add(layers.BatchNormalization())
#     model.add(layers.Dropout(0.25))
#     model.add(layers.Dense(128,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
#     model.add(layers.BatchNormalization())
#     model.add(layers.Dropout(0.25))
    
# #     model.add(layers.Dense(64,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
# #     model.add(layers.BatchNormalization())
# #     model.add(layers.Dropout(0.25))
# #     model.add(layers.Dense(64,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
# #     model.add(layers.BatchNormalization())
# #     model.add(layers.Dropout(0.25))
# #     model.add(layers.Dense(64,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
# #     model.add(layers.BatchNormalization())
# #     model.add(layers.Dropout(0.25))
    
    
    
#     model.add(layers.Dense(64,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
#     model.add(layers.BatchNormalization())
#     model.add(layers.Dropout(0.25))
#     model.add(layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
#     model.add(layers.BatchNormalization())
#     model.add(layers.Dropout(0.25))
# #     model.add(layers.Dense(8,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
# #     model.add(layers.BatchNormalization())
# #     model.add(layers.Dropout(0.25))
# #     model.add(layers.Dense(4,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
# #     model.add(layers.BatchNormalization())
# #     model.add(layers.Dropout(0.25))
#     model.add(layers.Dense(1, activation='softmax'))
    
#     model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer="adam")
    
#     return model

# modelNN=dl_nn()
# modelNN.summary()

In [31]:
# vectorizer=CountVectorizer(ngram_range=(1,3))
# xenc=vectorizer.fit_transform(x)
# xenc.shape

In [32]:
# type(xenc.toarray())

In [33]:
# callback=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=10,restore_best_weights=True)
# history=modelNN.fit(xenc,y_ord,batch_size=16,callbacks=[callback],epochs=200,shuffle=True)



In [34]:
# import matplotlib.pyplot as plt
# plt.plot(history.history['mae'])
# plt.plot(history.history['val_mae'])
# plt.legend(['train','test'],loc='lower right')
# plt.show()

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss']a)
# plt.legend(['train','test'],loc='upper right')
# plt.show()

In [35]:
# ypred=modelNN.predict(xtest)
# evaluate(ypred,ytest)

In [36]:
# #Fast Text FB

# def ft_data(x):
#     data=[]
#     for i in range(len(x)):
#         cat=re.sub(" ","_",x["Sub-category"][i])
#         s="__label__"+cat+" "+x["Article"][i]
#         data.append(s)
#     return data

# data=ft_data(train_data)
# data=pd.DataFrame({"text":data})
# data.to_csv("ft_data.csv",index=False)
# data.head()

In [37]:
# model = fasttext.train_supervised(input="./ft_data.csv")

In [38]:
# model.test("./ft_data.csv")

In [39]:
# # ##TFBERT

# import tensorflow as tf
# from transformers import TFAutoModelForSequenceClassification,AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=13)

# xtokens = tokenizer.batch_encode_plus(list(x),add_special_tokens=True,truncation=True,padding="max_length",max_length=512,return_attention_mask=True,return_tensors='tf')



In [40]:
# print(y_ord.shape)

In [41]:
# tfy=tf.keras.utils.to_categorical(y_ord, num_classes=13)

# data = tf.data.Dataset.from_tensor_slices((
#     dict(xtokens),
#     tfy
# ))

In [42]:

# model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer="adam")

# callback=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=10,restore_best_weights=True)
# history=model.fit(data,batch_size=32,callbacks=[callback],epochs=50,shuffle=True)



# **Predictions**

In [43]:

# en_stops=set(stopwords.words('english'))

# folder_test="../input/intermediate-nlp-mlbuzz/intermediate/dataset/test"
# test_data=pd.read_csv("../input/intermediate-nlp-mlbuzz/test_submission.csv")
# # test_data=pd.read_csv("../input/intermediate-nlp-mlbuzz/processed_test.csv")
# test_data.info()

In [44]:
# test_data["Sub_category"].iloc[6810]

In [45]:
# # reading the articles



en_stops=set(stopwords.words('english'))

folder_test="../input/intermediate-nlp-mlbuzz/intermediate/dataset/test"
test_data=pd.read_csv("../input/intermediate-nlp-mlbuzz/intermediate/dataset/test.csv")
test_data["Article"]=[""]*len(test_data)

mx=0

for i in range(len(test_data)):
    rel_path=test_data["File_name"].iloc[i]
    fpath=os.path.join(folder_test,rel_path)
    with open(fpath,'r') as f:
        art=f.read()
        proc_art=''
        
         #preprocessing: Removing stop words and special characters
        
        art=re.sub('[^A-Za-z\s]','',art)  #removing all non alphanumeric character with space
        
        for wd in art.split():
            wd=wd.strip()
            if wd not in en_stops:
                wd=wd.lower()
                wd=lemmaT.lemmatize(wd)
                if len(wd)>1:
                    proc_art+=wd+" "
                

        proc_art=proc_art.strip()
        test_data["Article"].iloc[i]=proc_art
        
        mx=max(mx,len(proc_art))


print("maxlen:",mx)

test_data.head()

maxlen: 6790


,File_name,Article
0,13938.txt,health canada review medical device ass safety...
1,13846.txt,in much preindustrial world guild system prote...
2,29876.txt,voting stake scania lcv dealer increasingly us...
3,8521.txt,it cause knowledge truth also object knowledge...
4,7729.txt,in additional electricity production solar ene...


In [46]:
test_data.to_csv("processed_test.csv",index=False)

In [47]:
test_data.head()

,File_name,Article
0,13938.txt,health canada review medical device ass safety...
1,13846.txt,in much preindustrial world guild system prote...
2,29876.txt,voting stake scania lcv dealer increasingly us...
3,8521.txt,it cause knowledge truth also object knowledge...
4,7729.txt,in additional electricity production solar ene...


In [48]:
# from transformers import pipeline
# from datetime import datetime
# classifier = pipeline("zero-shot-classification",model="facebook/bart-large-mnli")

# # test_data["Sub_category"]=['']*len(test_data)

# candidate_labels = ['Green Energy', 'Non-green Energy', 'Diagnostic', 'Pharma', 'Production', 'Toys', 'Food', 'Mechanical', 'Heavy Vehicles', 'Light Vehicles', 'Hardware', 'Software', 'Security']
# ypred=[]
# for i in range(6811,7945):
#     sequence_to_classify = test_data["Article"].iloc[i]
#     prediction=classifier(sequence_to_classify, candidate_labels)
#     labels=prediction['labels']
#     score=prediction['scores']
#     pred=np.argmax(score)
#     pred=labels[pred]
#     ypred.append(pred)
    
#     test_data["Sub_category"].iloc[i]=pred
    
#     if i%10==0:
#         print(i,datetime.now())
#         test_data.to_csv("test_submission.csv",index=False)
    

# sub_cat=test_data["Sub_category"] 
# industry=[cat_ind[i] for i in sub_cat]
# test_data=pd.DataFrame({"File_name":test_data["File_name"],"Industry":industry,"Sub_category":sub_cat})
# test_data.head()

In [49]:
# #Fast text FB

# test_data=pd.read_csv("../input/intermediate-nlp-mlbuzz/prepo_test.csv")
# test_ip=test_data["Article"]
# # test_ip.to_csv("test_input.csv",index=False)
# yhat=model.predict(list(test_ip))[0]
# sub_cat=[re.sub("_",' ',re.sub("__label__","",y[0])) for y in yhat]
# # industry=[cat_ind[i] for i in sub_cat]
# print(sub_cat[:5])


In [50]:

# sub_cat=ypred[:]
# industry=[cat_ind[i] for i in sub_cat]
# test_data.drop(columns=['Article'],inplace=True)
# test_data["Industry"]=industry
# test_data["Sub-category"]=sub_cat
# test_data.head()

In [51]:
# xtest=test_data["Article"]
# ycap=base_model.predict(xtest)
# sub_cat=enc_ord.inverse_transform(ycap.reshape(-1,1))
# sub_cat=[i[0] for i in sub_cat]
# print(sub_cat)
# industry=[cat_ind[i] for i in sub_cat]
# test_data.drop(columns=['Article'],inplace=True)
# test_data["Industry"]=industry
# test_data["Sub-category"]=sub_cat
# test_data.head()

In [52]:
# industry=[cat_ind[i] for i in sub_cat.reshape(-1)]
# test_data.drop(columns=['Article'],inplace=True)
# test_data["Industry"]=industry
# test_data["Sub-category"]=sub_cat
# test_data.head()

In [53]:
# test_data.to_csv("submission.csv",index=False)

In [54]:
# bert_enc=[]
# np_b_enc=np.empty((0,512,768))
# for i in range(leinverse_transform)):
#     enc=cls_enc[i]
#     b_enc=bert_model(**enc)[0]
# #     bert_enc.append(b_enc[0][0])
    

In [55]:
# from transformers import AutoModel

# # initialize cased BERT model
# bert = AutoModel.from_pretrained('bert-base-cased')

# input_ids = tf.keras.layers.Input(shape=(50,), name='input_ids', dtype='int32')
# mask = tf.keras.layers.Input(shape=(50,), name='attention_mask', dtype='int32')

# # we consume the last_hidden_state tensor from bert (discarding pooled_outputs)
# embeddings = bert(input_ids, attention_mask=mask)[0]

# X = tf.keras.layers.LSTM(64)(embeddings)
# X = tf.keras.layers.BatchNormalization()(X)
# X = tf.keras.layers.Dense(64, activation='relu')(X)
# X = tf.keras.layers.Dropout(0.1)(X)
# y = tf.keras.layers.Dense(13, activation='softmax', name='outputs')(X)

# # define input and output layers of our model
# model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# # freeze the BERT layer - otherwise we will be training 100M+ parameters...
# model.layers[2].trainable = False

In [56]:
# optimizer = tf.keras.optimizers.Adam(0.01)
# loss = tf.keras.losses.CategoricalCrossentropy()  # categorical = one-hot
# acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

# model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

# history = model.fit(train, validation_data=val, epochs=40)

In [57]:
# import numpy as np
# cls_enc=np.array(cls_enc)
# reshaped_cls_enc=cls_enc.reshape(cls_enc.shape[0],-1)
# np.savetext("bert_encoding.txt",reshaped_cls_enc)
# print(cls_enc.shape)
# print("probable shape:",31779,512,768)


In [58]:
#intermediate save
# train_data["Bert Encodings"]=cls_enc
# train_data.to_csv("bert_enc_proc_data.csv",index=False)

In [59]:
# x_gen=[enc[0] for enc in cls_enc]

In [60]:
# from sklearn.model_selection import train_test_split
# xtrain,xtest,ytrain,ytest=train_test_split(x_gen,y1hot,test_size=0.4,random_state=7)
# print(len(xtrain),len(xtest),len(ytrain),len(ytest))

In [61]:
# from sklearn.metrics import accuracy_score
# from xgboost.sklearn import XGBRegressor
# xgb=XGBRegressor()
# xgb.fit(xtrain,ytrain)
# ypred=xgb.predict(xtest)
# print(accuracy_score(ytest,ypred))
